In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 1. High Quality - 
# 2. Price

# 1. Price
# 2. Quality

In [2]:
data = pd.read_csv('jumia.csv.xls')
data.head()

,brand,current_price,old_price,discount,description,ratings,times_reviewed
0,Gionee,"₦ 48,000","₦ 73,630",-35%,Gionee K3 (M100) 4GB+64GB 6.22 Inch HD+ Androi...,70,10
1,Nokia,"₦ 59,520","₦ 119,660",-50%,"7， 5.2-inch (4GB, 64GB ROM) Android 7.1, 16MP ...",76,10
2,UMIDIGI,"₦ 25,990","₦ 29,990",-13%,"A3S Android 10 Global Band 3950mAh 5.7"" Smartp...",86,99
3,Nokia,"₦ 42,990","₦ 70,485",-39%,3.2 3GB RAM 32GB 6.26 Inch 13MP Camera Dual SI...,81,104
4,UMIDIGI,"₦ 25,990","₦ 29,990",-13%,"A3S Android 10 Global Band 3950mAh 5.7"" Smartp...",78,148


In [3]:
# checking for duplicate rows
data[data.duplicated()]

,brand,current_price,old_price,discount,description,ratings,times_reviewed


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 508 entries, 0 to 507
Data columns (total 7 columns):
brand             502 non-null object
current_price     508 non-null object
old_price         350 non-null object
discount          508 non-null object
description       508 non-null object
ratings           508 non-null int64
times_reviewed    508 non-null int64
dtypes: int64(2), object(5)
memory usage: 27.9+ KB


In [5]:
# cleaning the current price column
data['current_price'] = data['current_price'].str[1:].str.replace(',','').astype('int')

# cleaning the old price column
data['old_price'] = data['old_price'].str.replace('[₦\s,]', '')
data['old_price'] = data['old_price'].fillna(0).astype('int')

# cleaning the discount column
data['discount'] = data['discount'].str.strip('[% ₦]') 
data['discount'].loc[data['discount'].apply(lambda x: len(str(x)) > 4)] = 0

# cleaning the ratings column
data.ratings = data.ratings.apply(lambda x: 5*x/100)

C:\Users\User\Anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## Method 1

Leverages on the pandas regular expression methods (extractall) to find matching patterns, then those matching patterns will be returned as a series which is then used to create a column within the dataframe

In [6]:
# Creating a function that will help the data cleaning task
def clean_description(series, col):
    """
    clean_description(series, col)
    
    This function takes in a series and creates a column for that series within the an already existing dataframe. 
    It takes in two parameters:
    
    Parameters
    ----------
    series: pd.Series
        An already existing series object
    col: str
        Column name for the series object
    
    Returns
    ------
    Dataframe
    """
    # creating the column
    data[col] = np.NaN 
    
    # selecting the subset of our dataframe that are present in the series argument
    subset = data.loc[series.index] 
    
    subset[col] = series # fitting the subset of dataframe with values from the series
    data.loc[series.index] = subset # replacing the portion of the dataframe with our subset

## Method 2

Using the re library to match patterns that automatically return a dataframe that can then be concatenated to the existing dataframe. 

In [7]:
def clean(pattern, cols, names):
    import re
    """
    clean(pattern, cols, names)
    
    This function takes a pattern and returns a series or dataframe (depending on the way it matches). It takes in 
    three parameters:
        
    Parameters
    ----------
    pattern : str
        A regular expression that matches one or more options. 
            
    col : list
        A list that selects out the matched dataframe 
        
    names : list
        A list of strings that will replace the name of the columns within the dataframe
        
    Returns
    -------
    DataFrame
    """
    match = data['description'].apply(lambda x: re.findall(pattern, x))
    match_df = match.apply(lambda x: ','.join(x))
    match_df.columns = names
    return match_df.apply(lambda x: x.replace(['', None], np.NaN))

### Extracting the Rom and Ram

In [8]:
# Extracting the rom and ram
ram_rom = data['description'].str.extractall('(\d+GB)')

# selecting the rom from ram_rom
rom = ram_rom.xs(1, level = 'match')[0].str.replace('GB', '').astype('int')
clean_description(rom, 'rom') # creating the rom column

# selecting the ram from ram_rom
ram = ram_rom.xs(0, level = 'match')[0].str.replace('GB', '').astype('int')
clean_description(ram, 'ram') # creating the ram column

### Cleaning the Rom and Ram Columns

In [9]:
# Indexing out the data points where ram is bigger than rom 
mismatched = data[data['rom'] < data['ram']]
mismatched.rename(columns = {"ram":'rom', 'rom':'ram'}, inplace = True) # renaming the column to correct the abnormal

# rematching the values back to the dataframe
data.loc[mismatched.index] = mismatched

# the ram of a phone is rarely above 12
abnormal_ram = data[data.ram > 12]

# indexing out abnormal matches for the ram_rom column
ram_ = data['description'].str.extractall('([\d\+]+[GM]B)|([\d]+R[oaOA][mM])').xs(0, level = 'match')[0]
rom_ = data['description'].str.extractall('([\d\+]+[GM]B)|([\d]+R[oaOA][mM])').xs(1, level = 'match')

# getting the index of the mismatch
print('The index of the abnormal value within the rom_ series is:', 
      rom_[1][rom_[1] == '3RAM'].index[0])

# extracting values within the series ram_ that have unusual output
print('The index of the abnormal value within the ram_ series are:', 
      ram_.str.extractall('(\d\+\d*)').xs(0, level='match').index)

The index of the abnormal value within the rom_ series is: 114
The index of the abnormal value within the ram_ series are: Int64Index([22, 134, 161, 362, 440], dtype='int64')


C:\Users\User\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [10]:
# cleaning index 362 and 440
abnormal_ram['rom'].copy()[[362, 440, 114]] = 3

# cleaning index 22 and 161
abnormal_ram['rom'].copy()[[22, 161]] = 4

# cleaning index 134
abnormal_ram['rom'].copy()[134] = 2

# cleaning index 382
abnormal_ram['rom'].copy()[382] = 0.008

# cleaning index 482
abnormal_ram['rom'].copy()[482] = 0.542

# renaming the column to correct the abnormal
abnormal_ram.rename(columns = {"ram":'rom', 'rom':'ram'}, inplace = True) 

# rematching the values back to the dataframe
data.loc[abnormal_ram.index] = abnormal_ram

### Extracting the Battery, Camera quality, Sim, Screen dimension and O.S. Versions

In [11]:
# selecting out values that match the mah values of battery
battery = data['description'].str.extractall('([\d,]*[\s]*[Mm][aA][hH])').xs(0, level = 'match')[0].str.replace('([Mm][aA][hH])', '').str.replace(',', '').astype(float)

clean_description(battery, 'battery') # creating the battery column

In [12]:
# extracting the megapixel of phones
camera = data['description'].str.extractall('([\d.]*MP)').xs(0, level = 'match')[0].str.replace('MP','').str.strip('.').replace('', np.NaN).astype(float)

clean_description(camera, 'camera') # creating the battery column

In [13]:
# extracting the sim ports of phones
sims = data['description'].str.extractall('([\w]*[\s]*[Ss][iI][Mm])|([\w]*[\s]*[Ss]lots)').xs(0, level='match')
sims = sims[0].combine_first(sims[1])

clean_description(sims, 'sim port') # creating the sim port column

In [14]:
# extracting the screen dimension of phones
screen = data['description'].str.extractall('(\d*\.\d*[\'"])|([\d.\'"-]*[\s]*[Ii][nN][Cc][Hh])').xs(0, level='match')
screen = screen[0].combine_first(screen[1])
screen = screen.str.replace('[\'"iInch-]', '').astype(float)
clean_description(screen, 'screen_dim') # creating the screen dimension column

In [15]:
android = data['description'].str.extractall("([Aa]ndr[oi]*d [\s\d.-]{,3})").xs(0, level = 'match')[0].str.replace('[Aa]ndr[oi]*d[\s-]*', '').str.strip('.').replace('', np.NaN).astype(float)

clean_description(android, 'android_ver') # creating the Android version column

In [16]:
data.isnull().sum()

brand               6
current_price       0
old_price           0
discount            0
description         0
ratings             0
times_reviewed      0
rom                46
ram                52
battery           292
camera            177
sim port          303
screen_dim         67
android_ver       227
dtype: int64

### Cleaning Null Values

In [17]:
data['brand'].fillna(data['brand'].mode()[0], inplace = True)
data['rom'].fillna(data['rom'].median(), inplace = True)
data['ram'].fillna(data['ram'].median(), inplace = True) # mode and median are the same
data['battery'].fillna(data['battery'].median(), inplace = True)
data['camera'].fillna(data['camera'].median(), inplace = True) # median and mean are the same
data['sim port'].fillna(data['sim port'].mode()[0], inplace = True) # mode is for filling categorical data
data['screen_dim'].fillna(data['screen_dim'].median(), inplace = True) # median and mean are the same
data['android_ver'].fillna(data['android_ver'].median(), inplace = True)

In [18]:
data.isnull().sum()

brand             0
current_price     0
old_price         0
discount          0
description       0
ratings           0
times_reviewed    0
rom               0
ram               0
battery           0
camera            0
sim port          0
screen_dim        0
android_ver       0
dtype: int64

In [19]:
data.to_csv('Jumia_clean.csv', index = False)

## Analysis of the Data

In [20]:
brands = pd.get_dummies(data['brand'])

In [21]:
corr_data = pd.concat([brands, data], axis = 1)

In [22]:
corr_data.corr()['current_price'][corr_data.corr()['current_price'] >= 0.1].sort_values(ascending = False)[1:]

rom            0.594765
ram            0.564471
Samsung        0.396289
screen_dim     0.345991
Elephone       0.294111
old_price      0.293866
camera         0.292390
Oneplus        0.164135
android_ver    0.149642
battery        0.116763
Name: current_price, dtype: float64

In [23]:
corr_data.drop('current_price', axis = 1).corr()['old_price'][corr_data.corr()['old_price'] >= 0.1].sort_values(ascending = False)[1:]

ram           0.455032
rom           0.428723
Samsung       0.267196
battery       0.256389
screen_dim    0.222505
Blackview     0.150582
Name: old_price, dtype: float64

In [24]:
corr_data.drop('current_price', axis = 1).corr()['old_price'].abs()[corr_data.drop('current_price', axis = 1).corr()['old_price'].abs() >= 0.1].sort_values(ascending = False)[1:]

ram           0.455032
rom           0.428723
Samsung       0.267196
battery       0.256389
screen_dim    0.222505
Blackview     0.150582
Nokia         0.136979
Tecno         0.121520
Itel          0.121002
Name: old_price, dtype: float64

The correlation relationship show that the size of the rom and ram has the biggest biggest influence of the prices of phones. When the phones are initially listed (old prices), the samsung brand, battery and screeen dimension all have a low positive (less than 0.3) correlation and are in that order. 

Although the battery was one of the top factors to consider when the price of the phones were initially set, however, when the prices were set, they fell down the pecking order. Screen dimension, camera quality and android version became factors to consider before the battery. 

The correlation relationship also showed that itel, techno and nokia were negatively correlated with the listing price. That is once a phone is either itel, techno or nokia, the prices are often low. 

# data[(data['android_ver'] == 9) & (data['screen_dim'] > 6) & (data['rom'] > 100) & (data['camera'] > 16)]

using the rating column, check out products above and below the mean rating
most bought brand...is it the one with the highest rating?

In [25]:
data.head()

,brand,current_price,old_price,discount,description,ratings,times_reviewed,rom,ram,battery,camera,sim port,screen_dim,android_ver
0,Gionee,48000,73630,-35,Gionee K3 (M100) 4GB+64GB 6.22 Inch HD+ Androi...,3.50,10,64.0,4.0,5000.0,13.0,Dual SIM,6.22,8.1
1,Nokia,59520,119660,-50,"7， 5.2-inch (4GB, 64GB ROM) Android 7.1, 16MP ...",3.80,10,64.0,4.0,3000.0,16.0,Dual Sim,5.20,7.1
2,UMIDIGI,25990,29990,-13,"A3S Android 10 Global Band 3950mAh 5.7"" Smartp...",4.30,99,32.0,4.0,3950.0,13.0,Triple Slots,5.70,10.0
3,Nokia,42990,70485,-39,3.2 3GB RAM 32GB 6.26 Inch 13MP Camera Dual SI...,4.05,104,32.0,3.0,4000.0,13.0,Dual SIM,6.26,9.0
4,UMIDIGI,25990,29990,-13,"A3S Android 10 Global Band 3950mAh 5.7"" Smartp...",3.90,148,32.0,4.0,3950.0,13.0,Triple Slots,5.70,10.0


In [26]:
data.rom.unique()

array([ 64.,  32.,  16., 128., 256.,   8.,   4.])

In [27]:
data.ram.unique()

array([ 4.,  3.,  1.,  2.,  6., 12.,  8.])

In [28]:
ram_group = data.groupby('ram')

In [29]:
ram_group['ratings'].mean().sort_values(ascending = False)

ram
12.0    4.750000
6.0     4.378947
8.0     4.266667
1.0     4.207407
4.0     4.140952
3.0     4.104301
2.0     4.101667
Name: ratings, dtype: float64

In [30]:
ram_group['ratings'].std().sort_values(ascending = False)

ram
8.0     1.202712
3.0     1.010479
4.0     0.957138
6.0     0.870795
2.0     0.818844
1.0     0.673306
12.0    0.433013
Name: ratings, dtype: float64

In [31]:
rom_group = data.groupby('rom')

In [32]:
rom_group['ratings'].mean().sort_values(ascending = False)

rom
256.0    4.562500
128.0    4.466667
8.0      4.245000
64.0     4.180573
16.0     4.157407
32.0     4.096257
4.0      3.000000
Name: ratings, dtype: float64

In [33]:
rom_group['ratings'].std().sort_values(ascending = False)

rom
4.0      2.828427
32.0     1.040878
64.0     0.911266
128.0    0.795317
16.0     0.663377
8.0      0.628679
256.0    0.515388
Name: ratings, dtype: float64

In [34]:
ram_rom = data['ram'].apply(lambda x: str(x))+'ram' + ' ' + data['rom'].apply(lambda x: str(x)) + 'rom'

In [35]:
data['comb'] = ram_rom

In [36]:
data['rating_times'] = data['ratings'] * data['times_reviewed']

In [37]:
data.pivot_table(index = 'comb', values = ['rating_times', 'current_price']).sort_values('rating_times', ascending = False)

,current_price,rating_times
comb,,
1.0ram 16.0rom,36577.656250,310.895313
2.0ram 32.0rom,48645.880000,263.674000
1.0ram 8.0rom,25107.650000,139.622500
8.0ram 128.0rom,237473.875000,129.362500
4.0ram 128.0rom,141979.666667,114.433333
2.0ram 16.0rom,35482.028571,108.130000
4.0ram 64.0rom,77484.469565,105.783043
3.0ram 32.0rom,50573.424658,97.317808
4.0ram 32.0rom,60318.195402,86.216667


In [38]:
data['comb'].value_counts()

4.0ram 64.0rom      115
4.0ram 32.0rom       87
3.0ram 32.0rom       73
6.0ram 128.0rom      42
2.0ram 16.0rom       35
6.0ram 64.0rom       34
1.0ram 16.0rom       32
2.0ram 32.0rom       25
1.0ram 8.0rom        20
3.0ram 16.0rom       12
3.0ram 64.0rom        8
8.0ram 128.0rom       8
4.0ram 128.0rom       6
4.0ram 16.0rom        2
1.0ram 4.0rom         2
12.0ram 256.0rom      2
8.0ram 256.0rom       2
8.0ram 32.0rom        2
12.0ram 128.0rom      1
Name: comb, dtype: int64